In [1]:
import json
import requests
from loguru import logger
import dataset
import arrow
from time import sleep
import copy
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics
from sklearn.calibration import LabelEncoder
from sklearn.pipeline import Pipeline
import sqlalchemy
import pandas as pd
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import time
from sklearn.model_selection import learning_curve
matplotlib.use('TkAgg')


In [2]:

GAME_SESSION = 0

db = dataset.connect('sqlite:///db.db')

def get_match_X_data_for_game(game_session):
    #t.ground_truth_label_c
    q = 'select  t.sequence_num, t.ground_truth_label_c, p.response_classification_c '
    #q = 'select  t.ground_truth_label_c, p.response_classification_c '
    q = q + ' from llm_predictions p, text_blocks_table t where  p.cur_block_id_used = t.block_id and t.ground_truth_label_c = p.response_classification_c '
    q = q + ' and t.session_id = :v_parameters and llm_exp_name = "15 second v" order by t.session_id, t.sequence_num'     
    rs = db.engine.execute(sqlalchemy.text(q), v_parameters=game_session) 
    df_X = pd.DataFrame(rs)          
    return df_X

def get_non_match_X_data_for_game(game_session):
    #t.ground_truth_label_c
    q = 'select  t.sequence_num, t.ground_truth_label_c, p.response_classification_c '
    #q = 'select  t.ground_truth_label_c, p.response_classification_c '
    q = q + ' from llm_predictions p, text_blocks_table t where  p.cur_block_id_used = t.block_id and t.ground_truth_label_c <> p.response_classification_c'
    q = q + ' and t.session_id = :v_parameters and llm_exp_name = "15 second v7" order by t.session_id, t.sequence_num'     
    rs = db.engine.execute(sqlalchemy.text(q), v_parameters=game_session) 
    df_X = pd.DataFrame(rs)          
    return df_X

def get_base_X_data_for_game(game_session):
    #t.ground_truth_label_c
    q = 'select  t.sequence_num,  t.ground_truth_label_c base'
    #q = 'select  t.ground_truth_label_c, p.response_classification_c '
    q = q + ' from llm_predictions p, text_blocks_table t where  p.cur_block_id_used = t.block_id '
    q = q + ' and t.session_id = :v_parameters and llm_exp_name = "15 second v7" order by t.session_id, t.sequence_num'     
    rs = db.engine.execute(sqlalchemy.text(q), v_parameters=game_session) 
    df_X = pd.DataFrame(rs)          
    return df_X

np.random.seed(19680801)
x = np.random.randn(1000, 2)

n_bins = 4
#df_match = get_match_X_data_for_game(GAME_SESSION)
#df_non_match = get_non_match_X_data_for_game(GAME_SESSION)
base = get_base_X_data_for_game(GAME_SESSION)
#colors = ['blue', 'red']
#plt.hist(preds.to_numpy(), n_bins, density=True, histtype='bar', color=colors, label=['Actual Commercials', 'Predicted Commercials'])
#plt.legend(prop={'size': 10})
#plt.title('Commercials in Basketball Game by Timeline')
#fig_file = f"./output/c_freq.png"
#plt.savefig(fig_file)
#plt.clf()


  
# creating the dataset
#data = {'C':20, 'C++':15, 'Java':30, 
#        'Python':35}
x = base['sequence_num'].to_numpy()/4
y = base['base'].to_numpy()

#x = df_match['sequence_num'].to_numpy()/4
#y = df_match['ground_truth_label_c'].to_numpy()

#x_not = df_non_match['sequence_num'].to_numpy()/4
#y_not_act = df_non_match['ground_truth_label_c'].to_numpy()
#y_not_pred = df_non_match['response_classification_c'].to_numpy()

markers = [44/4, 172/4,266/4,328/4,463/4,610/4]
marker_values = [1,1,1,1,1,1]


C:\Users\bbfor\AppData\Local\Temp\ipykernel_32656\2236893426.py:31: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  rs = db.engine.execute(sqlalchemy.text(q), v_parameters=game_session)


In [6]:
  
fig = plt.figure(figsize = (10, 5))
 
# creating the bar plot

# MATCHES VIEW 

plt.bar(x, y, color ='green', 
        width = 0.1, alpha= .7)


#MISSES VIEW
plt.title("Commercials in a game - Actual")
#plt.bar(x_not, y_not_act, color ='orange', 
#        width = 0.1, alpha= .7)

#plt.bar(x_not, y_not_pred, color ='red', 
#        width = 0.1, alpha= .7)

plt.bar(markers, marker_values, color ='black', 
        width = 0.9,)

plt.xlabel("Game Timeline")
plt.ylabel("Commercials")

fig_file = f"./output/c_freq.png"
plt.savefig(fig_file)
plt.show()
plt.clf()